## Import Libraries and Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import time

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/104975 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompleteVZOR3Z/amazon_us_re…

Dataset amazon_us_reviews downloaded and prepared to ~/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.
<PrefetchDataset element_spec={'data': {'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'helpful_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'marketplace': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_category': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_parent': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_body': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_date': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_headline': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'star_rating': TensorSpec(shape=(), dtype=tf.int32, name=None), 'tota

In [3]:
df = tfds.as_dataframe(ds)

## Pre-processing

In [4]:
def to_sentiment(rating):
  rating = int(rating)
  if rating <= 2:
    return 'negative'
  elif rating == 3:
    return 'neutral'
  else:
    return 'positive'

# Mapping
df["Sentiment"] = df["data/star_rating"].apply(to_sentiment)
df['Sentiment'] = df['Sentiment'].map({'positive':2, 'negative':0, 'neutral': 1})

In [5]:
df['Customer_Review'] = df['data/review_body'].str.decode("utf-8")

In [6]:
df = df[["Customer_Review", "Sentiment"]]

In [7]:
n = 54975
df.drop(df.tail(n).index,
        inplace = True)

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Customer_Review'])
y = df.Sentiment

## Train/ Test Split

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Model Training

In [14]:
model = MultinomialNB()
start_time = time.time()
model.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04640913009643555 seconds ---


## Model Performance

In [11]:
pred = model.predict(X_valid)
acc = accuracy_score(y_valid, pred)
# f1 = f1_score(y_valid, pred)
conf = confusion_matrix(y_valid, pred)

print(f"Accuracy: {acc:.5f}")
# print(f"F1_Score: {f1:.5f}")
print(f"Confusion Matrix: {conf}")

Accuracy: 0.80110
Confusion Matrix: [[1785   53  555]
 [ 286   61  586]
 [ 376  133 6165]]


In [12]:
# Draw a table for Precision, Recall, F1-Score, Micro and Macro Averaging
labels = ['Negative', 'Neutral', 'Positive']
# Calculate overall accuracy
accuracy = np.sum(np.diagonal(conf)) / np.sum(conf)
print("Accuracy:", round(accuracy, 4))
print()

# Calculate precision, recall, and F1-score for each class
precision = np.empty(3)
recall = np.empty(3)
f1_score = np.empty(3)
for i in range(3):
    precision[i] = conf[i, i] / np.sum(conf[:, i])
    recall[i] = conf[i, i] / np.sum(conf[i, :])
    f1_score[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])
    print(labels[i])
    print("Precision:", round(precision[i], 4))
    print("Recall:", round(recall[i], 4))
    print("F1-score:", round(f1_score[i], 4))
    print()

# Calculate micro-averaged precision, recall, and F1-score
micro_precision = np.sum(np.diagonal(conf)) / np.sum(conf)
micro_recall = np.sum(np.diagonal(conf)) / np.sum(conf)
micro_f1_score = 2 * micro_precision * micro_recall / (micro_precision + micro_recall)
print("Micro-averaged precision:", round(micro_precision, 4))
print("Micro-averaged recall:", round(micro_recall, 4))
print("Micro-averaged F1-score:", round(micro_f1_score, 4))
print()

# Calculate macro-averaged precision, recall, and F1-score
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1_score = np.mean(f1_score)
print("Macro-averaged precision:", round(macro_precision, 4))
print("Macro-averaged recall:", round(macro_recall, 4))
print("Macro-averaged F1-score:", round(macro_f1_score, 4))

# Weighted-averaged Precision, Recall, and F1-score
num_instances = np.sum(conf)
weighted_precision = np.sum(precision * np.sum(conf, axis=1) / num_instances)
weighted_recall = np.sum(recall * np.sum(conf, axis=1) / num_instances)
weighted_f1_score = np.sum(f1_score * np.sum(conf, axis=1) / num_instances)
print("Weighted-averaged precision:", round(weighted_precision, 4))
print("Weighted-averaged recall:", round(weighted_recall, 4))
print("Weighted-averaged F1-score:", round(weighted_f1_score, 4))

Accuracy: 0.8011

Negative
Precision: 0.7295
Recall: 0.7459
F1-score: 0.7376

Neutral
Precision: 0.247
Recall: 0.0654
F1-score: 0.1034

Positive
Precision: 0.8438
Recall: 0.9237
F1-score: 0.882

Micro-averaged precision: 0.8011
Micro-averaged recall: 0.8011
Micro-averaged F1-score: 0.8011

Macro-averaged precision: 0.6068
Macro-averaged recall: 0.5783
Macro-averaged F1-score: 0.5743
Weighted-averaged precision: 0.7608
Weighted-averaged recall: 0.8011
Weighted-averaged F1-score: 0.7748


## Predict Sentiments using Model

In [13]:
labels = ['Negative', 'Neutral', 'Positive']
test_sentences = ["The product is not functioning at all. I definitely wouldn’t purchase it again."]
test_X  = vectorizer.transform(test_sentences)
predictions = model.predict(test_X)
print(predictions)
for i in predictions:
  print(labels[i])

[0]
Negative
